In [ ]:
import os
import sys
import pickle
import pandas as pd
import neurokit2 as nk
import matplotlib.pyplot as plt
from IPython.display import clear_output

sys.path.append("../")
from useful_functions.check_for_missing_data import check_for_missing_data
from useful_functions.physio_data.pd_dictionary import create_pd_dictionary
from useful_functions.physio_data.preprocess_physio_data import preprocess_physio_data
from useful_functions.physio_data.process_physio_timestamps import process_physio_timestamps

---

In [ ]:
driving_data_folder = "../../AdVitam/Exp2/Raw/Driving"
physio_data_folder = "../../AdVitam/Exp2/Raw/Physio/Txt"

In [ ]:
drivers_to_exclude = check_for_missing_data(driving_data_folder, physio_data_folder)
drivers_to_exclude.extend(["NST77", "NST11"]) #, "ST22", "NST87", "ST14", "ST12", "NST73", "ST10"])

---

In [ ]:
physio_timestamps = pd.read_csv(
    "../../AdVitam/Exp2/Preprocessed/Physio and Driving/timestamps_obstacles.csv"
)

In [ ]:
physio_timestamps = process_physio_timestamps(physio_timestamps, drivers_to_exclude)

---

In [ ]:
phsyiological_data_dictionary = create_pd_dictionary(physio_data_folder, drivers_to_exclude)

In [ ]:
phsyiological_data_dictionary = preprocess_physio_data(phsyiological_data_dictionary)

In [ ]:
# Find Average Beat/Minute
ST20 = phsyiological_data_dictionary["ST20"]
baseline_data = ST20['baseline']
testing_data = ST20['training']
driving_data = ST20['driving']

In [ ]:
# sliding window
window = 10000  # 10 seconds
step_size = 1000  # 1 second

hrv_data = pd.DataFrame()
for i in range(0, len(baseline_data), step_size):
    if i + window > len(baseline_data):
        break

    # Print progress
    print(f"{i}/{len(baseline_data)} = {i/len(baseline_data)*100}%")
    clear_output(wait=True)

    # Get window data
    window_data = baseline_data[i : i + window]

    # Compute HRV
    window_hrv = nk.hrv(baseline_data, sampling_rate=1000)

    # Add to dataframe
    window_hrv["Time"] = baseline_data["Time"].iloc[i + window]

    # Concatenate with hrv_data
    hrv_data = pd.concat([hrv_data, window_hrv], ignore_index=True)

In [ ]:
ST20 = phsyiological_data_dictionary["ST20"]["driving"]

In [ ]:
# plot the signals
fig, ax = plt.subplots(3, 1, figsize=(10, 10), sharex=True)
ax[0].plot(ST20["ECG_Clean"], label="ECG")
ax[0].set_title("ECG")
ax[1].plot(ST20["EDA_Clean"], label="EDA")
ax[1].set_title("EDA")
ax[2].plot(ST20["RSP_Clean"], label="RSP")
ax[2].set_title("RSP")
plt.show()

---

In [ ]:
os.makedirs("../../AdVitam_Processed/Exp2/Processed/Physio", exist_ok=True)

# Loop through physiological data and save as TXT
for driver in phsyiological_data_dictionary:
    # grab segmented data
    baseline_data = phsyiological_data_dictionary[driver]["baseline"]
    training_data = phsyiological_data_dictionary[driver]["training"]
    driving_data = phsyiological_data_dictionary[driver]["driving"]

    # Create the directory if it does not exist
    os.makedirs("../../AdVitam_Processed/Exp2/Processed/Physio/{driver}", exist_ok=True)

    # save the data as txt
    baseline_data.to_csv(f"../../AdVitam_Processed/Exp2/Processed/Physio/{driver}/{driver}_baseline.txt", index=False)
    training_data.to_csv(f"../../AdVitam_Processed/Exp2/Processed/Physio/{driver}/{driver}_training.txt", index=False)
    driving_data.to_csv(f"../../AdVitam_Processed/Exp2/Processed/Physio/{driver}/{driver}_driving.txt", index=False)


---

# ASIDE:

Participants with issues: ST22, NST87, ST14, ST12, NST73, ST10

---